<a href="https://colab.research.google.com/github/tituatgithub/chatbots_debating/blob/main/ai_debating_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate einops bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.5 MB/s eta 0:00:00


In [ ]:
import os
from huggingface_hub import login

login(os.getenv("HF_Token"))  # ✅ Gets the actual token value from env

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,  # Use float16/bfloat16 if you're on Colab T4/A100
)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
def get_model_response(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()  # Remove the prompt from response


In [ ]:
def run_debate(topic, rounds=3):
    model1_name = "LLaMA-Support"
    model2_name = "LLaMA-Oppose"
    previous_arguments = []

    print(f"\n🎯 DEBATE TOPIC: {topic}")

    for i in range(rounds):
        print(f"\n🟢 ROUND {i+1} — {model1_name}")
        prompt1 = f"Debate Topic: {topic}\nYou are an expert supporting the topic. Write your argument in a scholarly tone.\n"
        if previous_arguments:
            prompt1 += f"\nPrevious arguments:\n" + "\n".join(previous_arguments)
        arg1 = get_model_response(prompt1)
        print(f"\n{model1_name}: {arg1}")
        previous_arguments.append(f"{model1_name}: {arg1}")

        print(f"\n🔴 ROUND {i+1} — {model2_name}")
        prompt2 = f"Debate Topic: {topic}\nYou are an expert opposing the topic. Write a scholarly rebuttal.\n"
        prompt2 += f"\nOpponent's argument:\n{arg1}"
        if len(previous_arguments) > 1:
            prompt2 += f"\nPrevious arguments:\n" + "\n".join(previous_arguments[:-1])
        arg2 = get_model_response(prompt2)
        print(f"\n{model2_name}: {arg2}")
        previous_arguments.append(f"{model2_name}: {arg2}")


In [ ]:
# Try any topic you like
run_debate("god, and terrorism are they interlinked", rounds=8)



🎯 DEBATE TOPIC: god, and terrorism are they interlinked

🟢 ROUND 1 — LLaMA-Support

LLaMA-Support: The notion that god and terrorism are interlinked is a contentious issue that has sparked intense debate among scholars, policymakers, and the general public. While some argue that there is no direct correlation between religious beliefs and violent extremism, others contend that certain religious beliefs or interpretations can contribute to the perpetuation of terrorist acts.

From a historical perspective, it is evident that religion has been used as a justification for violence throughout human history. The Crusades, the Inquisition, and other instances of religiously motivated violence serve as stark reminders of the destructive potential of religious extremism. In modern times, the rise of Islamic fundamentalism has led to numerous terrorist attacks, with many perpetrators claiming to be acting in the name of their faith.

One of the primary arguments against linking god and terrori

OutOfMemoryError: CUDA out of memory. Tried to allocate 226.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 210.12 MiB is free. Process 18896 has 14.53 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 335.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)